In [1]:
import os 
import tfx 
import pipeline_helper
from tfx import v1 as tfx
from tfx_bsl.public import tfxio

SCHEMA_PIPELINE_NAME = "iris-schema-ma"
SCHEMA_PIPELINE_ROOT = os.path.join("pipelines", SCHEMA_PIPELINE_NAME)
SCHEMA_METADATA_PATH = os.path.join("metadata", SCHEMA_PIPELINE_NAME, "metadata.db")

PIPELINE_NAME = "iris-simple-ma"
PIPELINE_ROOT = os.path.join("pipelines", PIPELINE_NAME)
METADATA_PATH = os.path.join("metadata", PIPELINE_NAME, "metadata.db")

SERVING_MODEL_DIR = os.path.join("serving_model", PIPELINE_NAME)
DATA_ROOT = os.path.join("iris_data")

2022-05-04 12:04:22.648694: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-04 12:04:22.648720: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

In [3]:
# TODO Long, move to Transformer
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(data[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]])
# le = LabelEncoder()
# processed_data = pd.DataFrame(data=scaled_data, columns=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"])
# processed_data["Species"] = le.fit_transform(data["Species"].to_numpy().reshape(-1, 1))
# processed_data.to_csv("./iris_data/processed_iris_data.csv")

In [4]:
import urllib.request
import tempfile

_data_filepath = os.path.join("iris_data", "processed_iris_data.csv")

In [5]:
import pandas as pd
data = pd.read_csv(_data_filepath)
data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,-0.900681,1.032057,-1.341272,-1.312977,0
1,-1.143017,-0.124958,-1.341272,-1.312977,0
2,-1.385353,0.337848,-1.398138,-1.312977,0
3,-1.506521,0.106445,-1.284407,-1.312977,0
4,-1.021849,1.263460,-1.341272,-1.312977,0


# Create Schema

In [6]:
def create_schema_pipeline(
    pipeline_name,
    pipeline_root,
    data_root,
    metadata_path
):
    # Make sure directory has no other hidden files
    example_gen = tfx.components.CsvExampleGen(input_base=data_root)
    
    # Computes statistics for visualization and schema
    statistics_gen = tfx.components.StatisticsGen(
        examples=example_gen.outputs["examples"]
    )
    
    # Generate Schema 
    schema_gen = tfx.components.SchemaGen(
        statistics=statistics_gen.outputs["statistics"], infer_feature_shape=True
    )
    
    components = [
        example_gen,
        statistics_gen,
        schema_gen
    ]
    
    pipeline = tfx.dsl.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(metadata_path),
        components=components
    )    
    return pipeline

In [7]:
tfx.orchestration.LocalDagRunner().run(
    create_schema_pipeline(
        pipeline_name=SCHEMA_PIPELINE_NAME,
        pipeline_root=SCHEMA_PIPELINE_ROOT,
        data_root=DATA_ROOT,        
        metadata_path=SCHEMA_METADATA_PATH
    )
)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.example_gen.csv_example_gen.executor.Executor"
      }
    }
  }
}
executor_specs {
  key: "SchemaGen"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.schema_gen.executor.Executor"
    }
  }
}
executor_specs {
  key: "StatisticsGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.statistics_gen.executor.Executor"
      }
    }
  }
}
custom_driver_specs {
  key: "CsvExampleGen"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.example_gen.driver.FileBasedDriver"
    }
  }
}
metadata_connection_config {
  database_connection_config {
    sqlite {
      filename_uri: "metada

INFO:absl:Processing input csv data iris_data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Value type <class 'NoneType'> of key version in exec_properties is not supported, going to drop it
INFO:absl:Value type <class 'list'> of key _beam_pipeline_args in exec_properties is not supported, going to drop it
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 4 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "pipelines/iris-schema-ma/CsvExampleGen/examples/4"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:12071,xor_checksum:1651638713,sum_checksum:1651638713"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "iris-schema-ma:2022-05-04T12:04:26.672141:CsvExampleGen:examples:0"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
custom_propert

INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to pipelines/iris-schema-ma/StatisticsGen/statistics/5/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to pipelines/iris-schema-ma/StatisticsGen/statistics/5/Split-eval.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 5 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'statistics': [Artifact(artifact: uri: "pipelines/iris-schema-ma/StatisticsGen/statistics/5"
custom_properties {
  key: "name"
  value {
    string_value: "iris-schema-ma:2022-05-04T12:04:26.672141:StatisticsGen:statistics:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.6.1"
  }
}
, artifact_type: name: "ExampleStatistics"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
base_type: STATISTICS
)]}) for e

In [8]:
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    SCHEMA_METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:

    stat_gen_output = pipeline_helper.get_latest_artifacts(metadata_handler, SCHEMA_PIPELINE_NAME,
                                         'StatisticsGen')
    stats_artifacts = stat_gen_output[standard_component_specs.STATISTICS_KEY]
    
    pipeline_helper.visualize_artifacts(stats_artifacts)

INFO:absl:MetadataStore with DB connection initialized


# Create Training Pipeline

In [9]:
# with Metadata(metadata_connection_config) as metadata_handler:
# #         stat_gen_output = pipeline_helper.get_latest_artifacts(metadata_handler, SCHEMA_PIPELINE_NAME,
# #                                              'StatisticsGen')    
#     schema_gen_output = get_latest_artifacts(metadata_handler,
#                                            SCHEMA_PIPELINE_NAME, 'SchemaGen')
#     schema_artifacts = schema_gen_output[standard_component_specs.SCHEMA_KEY]
#     schema_artifacts[0].uri

In [10]:
import tensorflow_model_analysis as tfma
from tfx.orchestration.metadata import Metadata
from tfx.orchestration.metadata import sqlite_metadata_connection_config
from tfx.types import standard_component_specs

def create_pipeline(
    pipeline_name, 
    pipeline_root, 
    data_root, 
    module_file, 
    serving_model_dir, 
#     schema_path,
    metadata_path
): 
    # Make sure directory has no other hidden files
    example_gen = tfx.components.CsvExampleGen(input_base=data_root)
    
#     metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
#     SCHEMA_METADATA_PATH)

    with Metadata(metadata_connection_config) as metadata_handler:
#         stat_gen_output = pipeline_helper.get_latest_artifacts(metadata_handler, SCHEMA_PIPELINE_NAME,
#                                              'StatisticsGen')            
        statistics_artifacts = stat_gen_output[standard_component_specs.STATISTICS_KEY]
        
        schema_gen_output = pipeline_helper.get_latest_artifacts(metadata_handler,
                                               SCHEMA_PIPELINE_NAME, 'SchemaGen')
        schema_artifacts = schema_gen_output[standard_component_specs.SCHEMA_KEY]  
        
    statistics_gen = tfx.components.StatisticsGen(examples=example_gen.outputs["examples"])        
    
    # Import schema.
    schema_importer = tfx.dsl.Importer(
      source_uri=schema_artifacts[0].uri,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id('schema_importer')
    
    example_validator = tfx.components.ExampleValidator(
        statistics=statistics_gen.outputs["statistics"],
        schema=schema_importer.outputs["result"]
    )        
    
    
    trainer = tfx.components.Trainer(
        module_file=module_file,
        examples= example_gen.outputs["examples"],
        train_args=tfx.proto.TrainArgs(splits=["train"], num_steps=50),
        eval_args=tfx.proto.EvalArgs(splits=["train"], num_steps=5)
    )
    
    model_resolver = tfx.dsl.Resolver(
        strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
        # connects data producers and data consumers
        model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
        model_blessing=tfx.dsl.Channel(type=tfx.types.standard_artifacts.ModelBlessing)
    ).with_id("latest_blessed_model_resolver")    
    
    per_slice_threshold = tfma.PerSliceMetricThresholds(thresholds=[
        tfma.PerSliceMetricThreshold(
            slicing_specs=[tfma.SlicingSpec()],
            threshold=tfma.MetricThreshold(
                value_threshold=tfma.GenericValueThreshold(
                    lower_bound={"value": 0.6}
                ),
                change_threshold=tfma.GenericChangeThreshold(
                    direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                    absolute={"value": -1e-4}
                )
            )
        )
    ])    
    
    eval_config = tfma.EvalConfig(
        model_specs=[
            tfma.ModelSpec(label_key="Species")
        ],
        slicing_specs=[
            # Evaluate entire dataset 
            tfma.SlicingSpec(),
            # Evaluate per class
            tfma.SlicingSpec(feature_keys=["Species"])
        ],
        metrics_specs=[
            tfma.MetricsSpec(per_slice_thresholds={
                "sparse_categorical_accuracy": per_slice_threshold
            })
        ]
    )
    
    # Evaluator for checking 
    evaluator = tfx.components.Evaluator(
        examples=example_gen.outputs["examples"],
        model=trainer.outputs["model"],
        # Latest model used as baseline
        baseline_model=model_resolver.outputs["model"],
        eval_config=eval_config
    )
    
    pusher = tfx.components.Pusher(
        model=trainer.outputs["model"],
        # Only choose model if evaluation is better
        model_blessing=evaluator.outputs["blessing"],
        push_destination=tfx.proto.PushDestination(
            filesystem=tfx.proto.PushDestination.Filesystem(base_directory=serving_model_dir)
        )
    )
    
    components = [
        example_gen,
        statistics_gen,
        schema_importer,
        example_validator,
        trainer,        
        model_resolver,
        evaluator,
        pusher
    ]
    
    pipeline = tfx.dsl.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(metadata_path),
        components=components
    )
    
    return pipeline

In [11]:
tfx.orchestration.LocalDagRunner().run(
    create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file="iris_trainer.py", 
        serving_model_dir=SERVING_MODEL_DIR,
#         schema_path='Schema/Gen/schema/3',
        metadata_path=METADATA_PATH
    )
)

INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/home/long/Desktop/dev/learning/google/tfx/basics/iris_pj/iris_trainer.py' (including modules: ['pipeline_helper', 'iris_trainer', 'iris_util']).
INFO:absl:User module package has hash fingerprint version f2564edb09e7c9cae7b65cc7300aca3b252ebb246a2b4ad4bcaafb31c5fbd404.
INFO:absl:Executing: ['/home/long/anaconda3/envs/tfx/bin/python', '/tmp/tmpfy93q6tw/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpn51yhx0i', '--dist-dir', '/tmp/tmpo24yxbdg']
INFO:absl:Successfully built user code wheel distribution at 'pipelines/iris-simple-ma/_wheels/tfx_user_code_Trainer-0.0+f2564edb09e7c9cae7b65cc7300aca3b252ebb246a2b4ad4bcaafb31c5fbd404-py3-none-any.whl'; target user module is 'iris_trainer'.
INFO:absl:Full user module path is 'iris_trainer@pipeline

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying pipeline_helper.py -> build/lib
copying iris_trainer.py -> build/lib
copying iris_util.py -> build/lib
installing to /tmp/tmpn51yhx0i
running install
running install_lib
copying build/lib/pipeline_helper.py -> /tmp/tmpn51yhx0i
copying build/lib/iris_trainer.py -> /tmp/tmpn51yhx0i
copying build/lib/iris_util.py -> /tmp/tmpn51yhx0i
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpn51yhx0i/tfx_user_code_Trainer-0.0+f2564ed

INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Going to run a new execution 9
INFO:absl:Going to run a new execution: ExecutionInfo(execution_id=9, input_dict={}, output_dict=defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "pipelines/iris-simple-ma/CsvExampleGen/examples/9"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:12071,xor_checksum:1651638713,sum_checksum:1651638713"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "iris-simple-ma:2022-05-04T12:04:32.631252:CsvExampleGen:examples:0"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
, artifact_type: name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
base_type: DATASET
)]}), exec_properties

INFO:absl:Running as an importer node.
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: pipelines/iris-schema-ma/SchemaGen/schema/6, properties: {}, custom_properties: {}
INFO:absl:Component schema_importer is finished.
INFO:absl:Component StatisticsGen is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.statistics_gen.component.StatisticsGen"
    base_type: PROCESS
  }
  id: "StatisticsGen"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "iris-simple-ma"
      }
    }
  }
  contexts {
    type {
      name: "pipeline_run"
    }
    name {
      field_value {
        string_value: "2022-05-04T12:04:32.631252"
      }
    }
  }
  contexts {
    type {
      name: "node"
    }
    name {
      field_value {
        string_value: "iris-simple-ma.StatisticsGen"
      }
    }
  }
}
inputs {
  inputs {
    key: "examples"
    value {
      channe

INFO:absl:Component Trainer is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.trainer.component.Trainer"
    base_type: TRAIN
  }
  id: "Trainer"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "iris-simple-ma"
      }
    }
  }
  contexts {
    type {
      name: "pipeline_run"
    }
    name {
      field_value {
        string_value: "2022-05-04T12:04:32.631252"
      }
    }
  }
  contexts {
    type {
      name: "node"
    }
    name {
      field_value {
        string_value: "iris-simple-ma.Trainer"
      }
    }
  }
}
inputs {
  inputs {
    key: "examples"
    value {
      channels {
        producer_node_query {
          id: "CsvExampleGen"
        }
        context_queries {
          type {
            name: "pipeline"
          }
          name {
            field_value {
              string_value: "iris-simple-ma"
            }
          }
        }
        c

...

INFO:absl:udf_utils.get_fn {'example_splits': 'null', 'fairness_indicator_thresholds': 'null', 'eval_config': '{\n  "metrics_specs": [\n    {\n      "per_slice_thresholds": {\n        "sparse_categorical_accuracy": {\n          "thresholds": [\n            {\n              "slicing_specs": [\n                {}\n              ],\n              "threshold": {\n                "change_threshold": {\n                  "absolute": -0.0001,\n                  "direction": "HIGHER_IS_BETTER"\n                },\n                "value_threshold": {\n                  "lower_bound": 0.6\n                }\n              }\n            }\n          ]\n        }\n      }\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "Species"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "Species"\n      ]\n    }\n  ]\n}'} 'custom_eval_shared_model'
INFO:absl:Adding default baseline ModelSpec based on the candidate ModelSpec provided. The candidate model will

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Blessing result False written to pipelines/iris-simple-ma/Evaluator/blessing/14.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 14 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'evaluation': [Artifact(artifact: uri: "pipelines/iris-simple-ma/Evaluator/evaluation/14"
custom_properties {
  key: "name"
  value {
    string_value: "iris-simple-ma:2022-05-04T12:04:32.631252:Evaluator:evaluation:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.6.1"
  }
}
, artifact_type: name: "ModelEvaluation"
)], 'blessing': [Artifact(artifact: uri: "pipelines/iris-simple-ma/Evaluator/blessing/14"
custom_properties {
  key: "name"
  value {
    string_value: "iris-simple-ma:2022-05-04T12:04:32.631252:Evaluator:blessing:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    s

INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to pipelines/iris-simple-ma/ExampleValidator/anomalies/15/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to pipelines/iris-simple-ma/ExampleValidator/anomalies/15/Split-eval.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 15 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'anomalies': [Artifact(artifact: uri: "pipelines/iris-simple-ma/ExampleValidator/anomalies/15"
custom_properties {
  key: "name"
  value {
    string_value: "iris-simple-ma:2022-05-04T12:04:32.631252:ExampleValidator:anomalies:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.6.1"
  }
}
, artifact_type: name: "ExampleAnomalies"
properties {
  key: 

INFO:absl:Model on pipelines/iris-simple-ma/Evaluator/blessing/14 was not blessed by model validation
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 16 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'pushed_model': [Artifact(artifact: uri: "pipelines/iris-simple-ma/Pusher/pushed_model/16"
custom_properties {
  key: "name"
  value {
    string_value: "iris-simple-ma:2022-05-04T12:04:32.631252:Pusher:pushed_model:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.6.1"
  }
}
, artifact_type: name: "PushedModel"
)]}) for execution 16
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component Pusher is finished.


In [12]:
# Check validator
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:

    ex_comp = pipeline_helper.get_latest_artifacts(metadata_handler, PIPELINE_NAME,
                                         'ExampleValidator')
    anomalies_artifacts = ex_comp[standard_component_specs.ANOMALIES_KEY]
    
    pipeline_helper.visualize_artifacts(anomalies_artifacts)

INFO:absl:MetadataStore with DB connection initialized
